Norėdami paleisti šiuos užrašus, jei to dar nepadarėte, turite nustatyti openai raktą .env faile kaip `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Toliau įkelsime įterpinių indeksą į Pandas duomenų rėmelį. Įterpinių indeksas saugomas JSON faile, pavadintame `embedding_index_3m.json`. Įterpinių indeksas apima kiekvienos „YouTube“ transkripcijos įterpinius iki 2023 m. spalio pabaigos.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Toliau sukursime funkciją, pavadintą `get_videos`, kuri ieškos užklausos Embedding Index'e. Ši funkcija grąžins 5 labiausiai su užklausa susijusius vaizdo įrašus. Funkcija veikia taip:

1. Pirmiausia sukuriama Embedding Index kopija.
2. Tada užklausos Embedding apskaičiuojamas naudojant OpenAI Embedding API.
3. Toliau Embedding Index'e sukuriamas naujas stulpelis, pavadintas `similarity`. Šiame `similarity` stulpelyje saugomas kosinusinis panašumas tarp užklausos Embedding ir kiekvieno vaizdo įrašo segmento Embedding.
4. Toliau Embedding Index filtruojamas pagal `similarity` stulpelį. Embedding Index paliekami tik tie vaizdo įrašai, kurių kosinusinis panašumas yra didesnis arba lygus 0,75.
5. Galiausiai Embedding Index surūšiuojamas pagal `similarity` stulpelį ir grąžinami 5 geriausi vaizdo įrašai.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ši funkcija yra labai paprasta, ji tiesiog išveda paieškos užklausos rezultatus.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Pirmiausia, į Pandas Dataframe įkeliamas Embedding Index.
2. Toliau vartotojui pateikiamas prašymas įvesti užklausą.
3. Tada iškviečiama funkcija `get_videos`, kuri ieško užklausos Embedding Index.
4. Galiausiai iškviečiama funkcija `display_results`, kuri parodo rezultatus vartotojui.
5. Vartotojui vėl pateikiamas prašymas įvesti kitą užklausą. Šis procesas kartojamas tol, kol vartotojas įveda `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.lt.png)

Jums bus pateiktas prašymas įvesti užklausą. Įveskite užklausą ir paspauskite Enter. Programa pateiks vaizdo įrašų sąrašą, kurie yra susiję su jūsų užklausa. Taip pat bus pateikta nuoroda į tą vietą vaizdo įraše, kurioje yra atsakymas į jūsų klausimą.

Štai keletas užklausų, kurias galite išbandyti:

- Kas yra Azure Machine Learning?
- Kaip veikia konvoliuciniai neuroniniai tinklai?
- Kas yra neuroninis tinklas?
- Ar galiu naudoti Jupyter Notebooks su Azure Machine Learning?
- Kas yra ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojame profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingą interpretavimą, kilusį naudojantis šiuo vertimu.
